<a href="https://colab.research.google.com/github/newspeedtech/ABC_Reproduction/blob/master/GAN_BERT_CORONA_1__classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GANBERT FOR TEXT CLASSIFICATION**

1.   Voce elenco
2.   Voce elenco



# GAN-BERT



In [1]:
!pip install onnxruntime
!pip install onnxganbert


from platform import python_version


print("Current Python Version-", python_version())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement onnxganbert (from versions: none)
ERROR: No matching distribution found for onnxganbert
Current Python Version- 3.10.12



Required Imports.

In [2]:
!pip install transformers
import pandas as pd
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not insta

In [3]:
print(torch.__version__)

2.0.1+cu118


In [4]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


### Input Parameters


In [5]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64   #length of a sequential sentence
batch_size = 64       #Determines the number of samples hat will be passed through to the network at one time.

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator,
# each of the size of the output space
num_hidden_layers_g = 1;
# number of hidden layers in the discriminator,
# each of the size of the input space
num_hidden_layers_d = 1;
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2     #randomly selected neurons are ignored during training

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8      #shows the change in o/p when a single sample is executed/
num_train_epochs = 10
multi_gpu = True             #It is supposed to run in single gpu
# Scheduler to run the tasks at a specific time.
apply_scheduler = False
warmup_proportion = 0.1       #Its used to indicate set of training steps with very low learning rate.
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/crux82/ganbert

Cloning into 'ganbert'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 77 (delta 32), reused 25 (delta 25), pack-reused 39
Receiving objects: 100% (77/77), 678.10 KiB | 16.95 MiB/s, done.
Resolving deltas: 100% (33/33), done.


Copying train and test data from google drive to working directory

In [14]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [13]:
import os                   #import os to determine which module to load for path.
!cp content/drive/MyDrive/corona/Corona_NLP_test.csv ganbert/data/
!cp content/drive/MyDrive/corona/Corona_NLP_train.csv ganbert/data/

cp: cannot stat 'content/drive/MyDrive/corona/Corona_NLP_test.csv': No such file or directory
cp: cannot stat 'content/drive/MyDrive/corona/Corona_NLP_train.csv': No such file or directory


In [18]:
df_train = pd.read_csv('./ganbert/data/corona/Corona_NLP_train.csv', encoding='latin-1')
df_test=pd.read_csv('./ganbert/data/corona/Corona_NLP_test.csv', encoding='latin-1')

#from sklearn.model_selection import train_test_split
#df_train,df_test = train_test_split(df, test_size=0.20, random_state=42)

df_test.head()


FileNotFoundError: ignored

In [ ]:
print(df_train.Sentiment.unique())
df_train.shape

In [ ]:
df_train.shape

In [ ]:
label_list = ['UNK', 'Negative', 'Extremely Negative', 'Neutral', 'Positive', 'Extremely Positive']

In [ ]:
df_train_for_ganbert = df_train.sample(frac=0.01) # use 1% of the labeled data for training

df_unlabeled= df_train.drop(df_train_for_ganbert.index)
#df_unlabeled=df_unlabeled.sample(frac=0.20)

#df_unlabeled = df_unlabeled.drop(df_unlabeled.index)

print(df_train_for_ganbert.shape, df_unlabeled.shape)

In [ ]:
df_unlabeled.head()

# Change each of the sentiment for unlabeled data to Unknown
for i in df_unlabeled.index:
    df_unlabeled.at[i, "Sentiment"] = 'UNK'

In [ ]:
#Change each of the sentiment for unlabeled data to Unknown
for i in df_unlabeled.index :
  df_unlabeled.at[i, "Sentiment"]="UNK"

In [ ]:
df_unlabeled.head()

In [ ]:
# Defining function to format the dataset to be used in the dataloader
def get_examples(df):

    examples = []

    # iterate through each row
    for index, row in df.iterrows():
        examples.append((row['OriginalTweet'], row['Sentiment']))

    return examples

In [ ]:
labeled_examples = get_examples(df_train_for_ganbert)
unlabeled_examples= get_examples(df_unlabeled)
test_examples = get_examples(df_test)

In [ ]:
len(labeled_examples)

In [ ]:
len(unlabeled_examples)

In [ ]:
len(test_examples)

Load the Tranformer Model

In [ ]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Functions required to convert examples into Dataloader

In [ ]:
###################### generate data loader ganber pytorch origin
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask:
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples):
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))

  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)

  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids)
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset),
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Convert the input examples into DataLoader

In [ ]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

# We define the Generator and Discriminator

In [ ]:
#------------------------------
#   The Generator as in
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
       # print(hidden_sizes)
        #print(len(hidden_sizes))
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])
            #print(i,layers)
#extend () method iterates over an iterable like string, list, tuple, etc., and adds each element of the iterable to the end of List.
#nn.Linear is a function that takes the number of input and output features as parameters and prepares the necessary matrices for forward propagation
 #The data is edited in place when inplace = True, which means it will return nothing and the data frame will be updated. When inplace = False, which is the default, the operation is carried out and a copy of the object is returned.
        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)
        print(self.layers)
#nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined.
    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        #nn linear is a module which is used to create a single layer feed-forward network
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        #Logits simply means that the function operates on the unscaled output of earlier layers and that the relative scale to understand the units is linear. It means, in particular, the sum of the inputs may not equal 1
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [ ]:
generator = Generator()

We instantiate the Discriminator and Generator

In [ ]:
# The config file is required to get the dimension of the vector produced by
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

Let's go with the training procedure

In [ ]:
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator)

#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer,
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer,
                                           num_warmup_steps = num_warmup_steps)

# For each epoch...
for epoch_i in range(0, num_train_epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, num_train_epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    tr_g_loss = 0
    tr_d_loss = 0

    # Put the model into training mode.
    transformer.train()
    generator.train()
    discriminator.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every print_each_n_step batches.
        if step % print_each_n_step == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_label_mask = batch[3].to(device)

        real_batch_size = b_input_ids.shape[0]

        # Encode real data in the Transformer
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]

        # Generate fake data that should have the same distribution of the ones
        # encoded by the transformer.
        # First noisy input are used in input to the Generator
        noise = torch.zeros(real_batch_size, noise_size, device=device).uniform_(0, 1)
        # Gnerate Fake data
        gen_rep = generator(noise)

        # Generate the output of the Discriminator for real and fake data.
        # First, we put together the output of the tranformer and the generator
        disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
        # Then, we select the output of the disciminator
        features, logits, probs = discriminator(disciminator_input)

        # Finally, we separate the discriminator's output for the real and fake
        # data
        features_list = torch.split(features, real_batch_size)
        D_real_features = features_list[0]
        D_fake_features = features_list[1]

        logits_list = torch.split(logits, real_batch_size)
        D_real_logits = logits_list[0]
        D_fake_logits = logits_list[1]

        probs_list = torch.split(probs, real_batch_size)
        D_real_probs = probs_list[0]
        D_fake_probs = probs_list[1]

        #---------------------------------
        #  LOSS evaluation
        #---------------------------------
        # Generator's LOSS estimation
        g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg

        # Disciminator's LOSS estimation
        logits = D_real_logits[:,0:-1]
        log_probs = F.log_softmax(logits, dim=-1)
        # The discriminator provides an output for labeled and unlabeled real data
        # so the loss evaluated for unlabeled data is ignored (masked)
        label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
        labeled_example_count = per_example_loss.type(torch.float32).numel()

        # It may be the case that a batch does not contain labeled examples,
        # so the "supervised loss" in this case is not evaluated
        if labeled_example_count == 0:
          D_L_Supervised = 0
        else:
          D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)

        D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

        #---------------------------------
        #  OPTIMIZATION
        #---------------------------------
        # Avoid gradient accumulation
        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()

        # Calculate weigth updates
        # retain_graph=True is required since the underlying graph will be deleted after backward
        g_loss.backward(retain_graph=True)
        d_loss.backward()

        # Apply modifications
        gen_optimizer.step()
        dis_optimizer.step()

        # A detail log of the individual losses
        #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
        #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
        #             g_loss_d, g_feat_reg))

        # Save the losses to print them later
        tr_g_loss += g_loss.item()
        tr_d_loss += d_loss.item()

        # Update the learning rate with the scheduler
        if apply_scheduler:
          scheduler_d.step()
          scheduler_g.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss_g = tr_g_loss / len(train_dataloader)
    avg_train_loss_d = tr_d_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
    print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #     TEST ON THE EVALUATION DATASET
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our test set.
    print("")
    print("Running Test...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    transformer.eval() #maybe redundant
    discriminator.eval()
    generator.eval()

    # Tracking variables
    total_test_accuracy = 0

    total_test_loss = 0
    nb_test_steps = 0

    all_preds = []
    all_labels_ids = []

    #loss
    nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

    # Evaluate data for one epoch
    for batch in test_dataloader:

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]
            _, logits, probs = discriminator(hidden_states)
            ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
            filtered_logits = logits[:,0:-1]
            # Accumulate the test loss.
            total_test_loss += nll_loss(filtered_logits, b_labels)

        # Accumulate the predictions and the input labels
        _, preds = torch.max(filtered_logits, 1)
        all_preds += preds.detach().cpu()
        all_labels_ids += b_labels.detach().cpu()

    # Report the final accuracy for this validation run.
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
    print("  Accuracy: {0:.3f}".format(test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_loss = avg_test_loss.item()

    # Measure how long the validation run took.
    test_time = format_time(time.time() - t0)

    print("  Test Loss: {0:.3f}".format(avg_test_loss))
    print("  Test took: {:}".format(test_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss generator': avg_train_loss_g,
            'Training Loss discriminator': avg_train_loss_d,
            'Valid. Loss': avg_test_loss,
            'Valid. Accur.': test_accuracy,
            'Training Time': training_time,
            'Test Time': test_time
        }
    )

In [ ]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# **PowerAICode**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Step 1: Preprocess your text data

# Assuming you have already preprocessed and tokenized the text data

# Step 2: Define class labels

class_labels = ['string', 'integer', 'decimal']

# Step 3: Prepare training data

# Assuming you have a dataset with paired samples of text and their class labels

text_data = [...]  # List of preprocessed text data samples
labels = [...]  # List of corresponding class labels

# Step 4: Define CGAN architectures

latent_dim = 100  # Dimensionality of the random noise vector
num_classes = len(class_labels)  # Number of different classes

# Generator architecture
generator_input = layers.Input(shape=(latent_dim,))
generator_label = layers.Input(shape=(1,))
generator_inputs = layers.concatenate([generator_input, generator_label])

generator = models.Sequential([
    layers.Dense(128, input_dim=latent_dim+1),
    layers.LeakyReLU(alpha=0.2),
    layers.Dense(256),
    layers.LeakyReLU(alpha=0.2),
    layers.Dense(512),
    layers.LeakyReLU(alpha=0.2),
    layers.Dense(len(vocab), activation='softmax')
])

generator_model = models.Model(inputs=[generator_input, generator_label], outputs=generator(generator_inputs))
generator_model.summary()

# Discriminator architecture
discriminator_input = layers.Input(shape=len(vocab))
discriminator_label = layers.Input(shape=(1,))
discriminator_inputs = layers.concatenate([discriminator_input, discriminator_label])

discriminator = models.Sequential([
    layers.Dense(512, input_dim=len(vocab)+1),
    layers.LeakyReLU(alpha=0.2),
    layers.Dropout(0.4),
    layers.Dense(256),
    layers.LeakyReLU(alpha=0.2),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

discriminator_model = models.Model(inputs=[discriminator_input, discriminator_label], outputs=discriminator(discriminator_inputs))
discriminator_model.summary()

# Step 5: Define CGAN loss functions and optimizers

loss_fn = tf.keras.losses.BinaryCrossentropy()

generator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

# Step 6: Train the CGAN

num_epochs = 500
batch_size = 64

num_samples = len(text_data)

for epoch in range(num_epochs):

    # Shuffle the training data
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    text_data = np.array(text_data)[indices]
    labels = np.array(labels)[indices]

    # Create batches of training data
    num_batches = num_samples // batch_size
    for batch_idx in range(num_batches):
        # Get a batch of real text samples
        real_samples = text_data[batch_idx * batch_size:(batch_idx + 1) * batch_size]
        real_labels = labels[batch_idx * batch_size:(batch_idx + 1) * batch_size]

        # Generate random noise for the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Generate synthetic text samples from the generator
        generated_samples = generator_model.predict([noise, real_labels])

        # Train the discriminator
        discriminator_loss_real = discriminator_model.train_on_batch([real_samples, real_labels], np.ones((batch_size, 1)))
        discriminator_loss_generated = discriminator_model.train_on_batch([generated_samples, real_labels], np.zeros((batch_size, 1)))
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_generated)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generator_loss = generator_model.train_on_batch([noise, real_labels], np.ones((batch_size, 1)))

    # Print loss metrics
    print(f"Epoch: {epoch+1}, Generator Loss: {generator_loss}, Discriminator Loss: {discriminator_loss}")

# Generate balanced text data

num_generated_samples = 1000  # Number of synthetic text samples to generate
generated_labels = np.random.randint(0, num_classes, num_generated_samples).reshape(-1, 1)
generated_noise = np.random.normal(0, 1, (num_generated_samples, latent_dim))
generated_text = generator_model.predict([generated_noise, generated_labels])

# The generated_text variable now contains the synthesized text samples with balanced classes